# Analysis of Trajectories

## Import Libraries

In [ ]:
!git clone https://github.com/clinicalml/gumbel-max-scm.git

Cloning into 'gumbel-max-scm'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 110
Receiving objects: 100% (113/113), 1.48 MiB | 13.62 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
#Enable importing code from parent directory
import os, sys
simulator_path = os.path.abspath('./gumbel-max-scm')
sys.path.insert(1, simulator_path)

In [ ]:
!pip install pymdptoolbox

  Preparing metadata (setup.py) ... done
  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-py3-none-any.whl size=25656 sha256=c19c7d694bb8f5b64ae5e43c95f19c742351c604530ff326d3ae5c50df0f4ae4
  Stored in directory: /root/.cache/pip/wheels/2b/e7/c7/d7abf9e309f3573a934fed2750c70bd75d9e9d901f7f16e183
Successfully built pymdptoolbox


**IMPORTANT NOTE:** At this stage, to reproduce our experiments, one must modify line 38 of `gumbel-max-scm/sepsisSimDiabetes/DataGenerator.py` so that it reads:

```
mdp = MDP(init_state_idx=%state%,
          policy_array=policy, policy_idx_type=policy_idx_type,
          p_diabetes=p_diabetes)

```

We have essentially set the initial state to a fixed value so that we may estimate the Q-function from that state. Additionally, line 58 of the same file must be modified to:

```
mdp.state = mdp.get_new_state(state_idx = %state%)
```

In [ ]:
import numpy as np
import cf.counterfactual as cf
import cf.utils as utils
import pandas as pd
import pickle
import itertools as it
from tqdm import tqdm_notebook as tqdm
from scipy.linalg import block_diag

# Sepsis Simulator code
from sepsisSimDiabetes.State import State
from sepsisSimDiabetes.Action import Action
from sepsisSimDiabetes.DataGenerator import DataGenerator
import sepsisSimDiabetes.MDP as simulator

import mdptoolboxSrc.mdp as mdptools

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Set up Variables for Policy and Data

In [ ]:
import numpy as np
SEED = 1
np.random.seed(SEED)
NSIMSAMPS = 100000  # Samples to draw from the simulator
NSTEPS = 20  # Max length of each trajectory
NCFSAMPS = 5  # Counterfactual Samples per observed sample
DISCOUNT_Pol = 0.99 # Used for computing optimal policies
DISCOUNT = 1 # Used for computing actual reward
PHYS_EPSILON = 0.05 # Used for sampling using physician pol as eps greedy

PROB_DIAB = 0.0

# Option 1: Use bootstrapping w/replacement on the original NSIMSAMPS to estimate errors
USE_BOOSTRAP=True
N_BOOTSTRAP = 100

# Option 2: Use repeated sampling (i.e., NSIMSAMPS fresh simulations each time) to get error bars;
# This is done in the appendix of the paper, but not in the main paper
N_REPEAT_SAMPLING = 1

# These are properties of the simulator, do not change
n_actions = Action.NUM_ACTIONS_TOTAL
n_components = 2

# These are added as absorbing states
n_states_abs = State.NUM_OBS_STATES + 2
discStateIdx = n_states_abs - 1
deadStateIdx = n_states_abs - 2

In [ ]:
import zipfile
with zipfile.ZipFile("gumbel-max-scm/data/diab_txr_mats-replication.zip", 'r') as zip_ref:
    zip_ref.extractall("gumbel-max-scm/data")

In [ ]:
# Get the transition and reward matrix from file
with open("gumbel-max-scm/data/diab_txr_mats-replication.pkl", "rb") as f:
    mdict = pickle.load(f)

tx_mat = mdict["tx_mat"]
r_mat = mdict["r_mat"]
p_mixture = np.array([1 - PROB_DIAB, PROB_DIAB])

In [ ]:
from scipy.linalg import block_diag

tx_mat_full = np.zeros((n_actions, State.NUM_FULL_STATES, State.NUM_FULL_STATES))
r_mat_full = np.zeros((n_actions, State.NUM_FULL_STATES, State.NUM_FULL_STATES))

for a in range(n_actions):
    tx_mat_full[a, ...] = block_diag(tx_mat[0, a, ...], tx_mat[1, a,...])
    r_mat_full[a, ...] = block_diag(r_mat[0, a, ...], r_mat[1, a, ...])

In [ ]:
fullMDP = cf.MatrixMDP(tx_mat_full, r_mat_full)
fullPol = fullMDP.policyIteration(discount=DISCOUNT_Pol, eval_type=1)

In [ ]:
hr_state_mapping = ['L ', 'N ', 'H ']
sbp_state_mapping = ['L ', 'N ', 'H ']
o2_state_mapping = ['L ', 'N ']
glu_state_mapping = ['VL', 'L ', 'N ', 'H ', 'VH']
abx_state_mapping = ['Of', 'On']
vaso_state_mapping = ['Of', 'On']
vent_state_mapping = ['Of', 'On']
diab_state_mapping = ['No', 'Ye']

In [ ]:
def pretty_print(trajectory_str, diabetic_idx):

  remove_brackets = trajectory_str[1:-1]
  states = remove_brackets.split(',')
  HRs = []
  SBPs = []
  POs = []
  GL = []
  AB = []
  VA = []
  VE = []
  for state_idx in states:
    stateObj = State(state_idx = int(float(state_idx)), diabetic_idx=diabetic_idx)
    vec = stateObj.get_state_vector()
    HRs.append(hr_state_mapping[vec[0]])
    SBPs.append(sbp_state_mapping[vec[1]])
    POs.append(o2_state_mapping[vec[2]])
    GL.append(glu_state_mapping[vec[3]])
    AB.append(abx_state_mapping[vec[4]])
    VA.append(vaso_state_mapping[vec[5]])
    VE.append(vent_state_mapping[vec[6]])

  print(f'HRs : {HRs}')
  print(f'SBPs: {SBPs}')
  print(f'POs : {POs}')
  print(f'GL  : {GL}')
  print(f'AB  : {AB}')
  print(f'VA  : {VA}')
  print(f'VE  : {VE}')

## Perform Analysis on Physician Policy

Generate Behaviour Policy Data

In [ ]:
dgen = DataGenerator()
states, actions, lengths, rewards, diab, emp_tx_totals, emp_r_totals = dgen.simulate(
    NSIMSAMPS, NSTEPS, policy=fullPol, policy_idx_type='full',
    p_diabetes=PROB_DIAB, use_tqdm=False) #True, tqdm_desc='Behaviour Policy Simulation')

obs_samps = utils.format_dgen_samps(
    states, actions, rewards, diab, NSTEPS, NSIMSAMPS)

Convert data into array format

In [ ]:
time = np.arange(NSTEPS)
times = np.stack(axis=0, arrays=[time]*NSIMSAMPS)
times = times[..., np.newaxis]

nf_transitions_b = np.concatenate((times, states[:, 0:NSTEPS, :], actions, rewards, states[:, 1:, :]), axis=2)
nf_transitions_b.shape

(100000, 20, 5)

In [ ]:
import numpy as np
import gc

def hashgen(arr):
  return np.array2string(arr[:, 4], precision=2, separator=',', suppress_small=True)

def analyse_trajectories(nf, NUM_EPISODES, EPISODE_LENGTH=20):

  #Obtain dimensions
  N, T = nf.shape[0], nf.shape[1]

  counts = {}
  last_states = {}

  for n in range(NUM_EPISODES):
      nf_tr = nf[n, :EPISODE_LENGTH, :]

      last_state_id = EPISODE_LENGTH - 1
      try:
        last_state_id = np.where(nf_tr[:, 4] == -1)[0][0] - 1
      except:
        pass

      last_state = nf_tr[last_state_id, 4]

      hash = hashgen(nf_tr[:last_state_id+1, :])
      if hash not in counts.keys():
        counts[hash] = 1
      else:
        counts[hash] += 1
        last_states[hash] = last_state

  return counts, last_states, len(counts.keys())

In [ ]:
traj_count, last_states, num = analyse_trajectories(nf_transitions_b, 100000)

print(num)

counts = list(traj_count.values())
trajs = list(traj_count.keys())
for (seq, id) in enumerate(np.flip(np.argsort(counts)[-50:])):
  last_state_num = last_states[trajs[id]]
  test_state = State(state_idx = last_state_num, diabetic_idx=int(PROB_DIAB))
  num_abnormal = test_state.get_num_abnormal()
  print(f'{seq+1}. Occurrences: {counts[id]}. Trajectory: {trajs[id]} Num abnormal of last state: {num_abnormal}')
  pretty_print(trajs[id], int(PROB_DIAB))

72031
1. Occurrences: 10918. Trajectory: [437.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 1345. Trajectory: [429.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 1317. Trajectory: [413.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
4. Occurrences: 1131. Trajectory: [277.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
5. Occurrences: 202. Trajectory: [389.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of']
6. Occurrences: 180. Trajectory: [389.,388.,376.] Num abnormal of last state: 0
HRs : ['N 

In [ ]:
int(PROB_DIAB)

0

## Perform Analysis on Random Policy

In [ ]:
#Set up fully random policy
randPol = np.ones(fullPol.shape)/(fullPol.shape[1])

Generate Behaviour Policy Data

In [ ]:
dgen = DataGenerator()
states, actions, lengths, rewards, diab, emp_tx_totals, emp_r_totals = dgen.simulate(
    NSIMSAMPS, NSTEPS, policy=randPol, policy_idx_type='full',
    p_diabetes=PROB_DIAB, use_tqdm=False) #True, tqdm_desc='Behaviour Policy Simulation')

obs_samps = utils.format_dgen_samps(
    states, actions, rewards, diab, NSTEPS, NSIMSAMPS)

Convert data into array format

In [ ]:
time = np.arange(NSTEPS)
times = np.stack(axis=0, arrays=[time]*NSIMSAMPS)
times = times[..., np.newaxis]

nf_transitions_b = np.concatenate((times, states[:, 0:NSTEPS, :], actions, rewards, states[:, 1:, :]), axis=2)
nf_transitions_b.shape

(100000, 20, 5)

In [ ]:
import numpy as np
import gc

def hashgen(arr):
  return np.array2string(arr[:, 4], precision=2, separator=',', suppress_small=True)

def analyse_trajectories(nf, NUM_EPISODES, EPISODE_LENGTH=20):

  #Obtain dimensions
  N, T = nf.shape[0], nf.shape[1]

  counts = {}
  last_states = {}

  for n in range(NUM_EPISODES):
      nf_tr = nf[n, :EPISODE_LENGTH, :]

      last_state_id = EPISODE_LENGTH - 1
      try:
        last_state_id = np.where(nf_tr[:, 4] == -1)[0][0] - 1
      except:
        pass

      last_state = nf_tr[last_state_id, 4]

      hash = hashgen(nf_tr[:last_state_id+1, :])
      if hash not in counts.keys():
        counts[hash] = 1
      else:
        counts[hash] += 1
        last_states[hash] = last_state

  return counts, last_states, len(counts.keys())

In [ ]:
traj_count, last_states, num = analyse_trajectories(nf_transitions_b, 100000)

print(num)

counts = list(traj_count.values())
trajs = list(traj_count.keys())
for (seq, id) in enumerate(np.flip(np.argsort(counts)[-50:])):
  last_state_num = last_states[trajs[id]]
  test_state = State(state_idx = last_state_num, diabetic_idx=int(PROB_DIAB))
  num_abnormal = test_state.get_num_abnormal()
  print(f'{seq+1}. Occurrences: {counts[id]}. Trajectory: {trajs[id]} Num abnormal of last state: {num_abnormal}')
  pretty_print(trajs[id], int(PROB_DIAB))

33957
1. Occurrences: 12058. Trajectory: [438.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
2. Occurrences: 11242. Trajectory: [434.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
3. Occurrences: 7393. Trajectory: [432.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
4. Occurrences: 4213. Trajectory: [436.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
5. Occurrences: 3677. Trajectory: [439.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['On']
VE  : ['On']
6. Occurrences: 3348. Trajectory: [435.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
7. Occurrences: 3202

In [ ]:
int(PROB_DIAB)

1

## Record Data for Physician Policy (top 50 Trajectories over 100000 episodes)

### State 136 With Diabetes

In [ ]:
'''
48690
1. Occurrences: 2755. Trajectory: [225.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 2738. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 2723. Trajectory: [65.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
4. Occurrences: 2711. Trajectory: [209.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
5. Occurrences: 852. Trajectory: [129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
6. Occurrences: 770. Trajectory: [137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
7. Occurrences: 769. Trajectory: [129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
8. Occurrences: 765. Trajectory: [145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
9. Occurrences: 762. Trajectory: [137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
10. Occurrences: 754. Trajectory: [137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
11. Occurrences: 736. Trajectory: [137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
12. Occurrences: 730. Trajectory: [145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
13. Occurrences: 670. Trajectory: [ 57.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
14. Occurrences: 613. Trajectory: [145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
15. Occurrences: 597. Trajectory: [129.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'VL']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
16. Occurrences: 586. Trajectory: [129., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['L ', 'VL']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
17. Occurrences: 576. Trajectory: [145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
18. Occurrences: 566. Trajectory: [217.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
19. Occurrences: 503. Trajectory: [217.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
20. Occurrences: 400. Trajectory: [377.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
21. Occurrences: 323. Trajectory: [145.,153., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
22. Occurrences: 308. Trajectory: [129.,121.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
23. Occurrences: 299. Trajectory: [129.,121., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
24. Occurrences: 268. Trajectory: [145.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
25. Occurrences: 261. Trajectory: [137.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
26. Occurrences: 243. Trajectory: [137.,137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
27. Occurrences: 234. Trajectory: [129.,129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
28. Occurrences: 233. Trajectory: [385.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
29. Occurrences: 230. Trajectory: [137.,129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
30. Occurrences: 224. Trajectory: [377.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
31. Occurrences: 219. Trajectory: [137.,129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
32. Occurrences: 218. Trajectory: [137.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
33. Occurrences: 215. Trajectory: [137.,137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
34. Occurrences: 214. Trajectory: [129.,129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
35. Occurrences: 212. Trajectory: [137.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
36. Occurrences: 209. Trajectory: [369.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
37. Occurrences: 206. Trajectory: [377.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
38. Occurrences: 206. Trajectory: [385.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
39. Occurrences: 203. Trajectory: [137.,137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
40. Occurrences: 201. Trajectory: [369.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
41. Occurrences: 200. Trajectory: [145.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
42. Occurrences: 199. Trajectory: [145.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
43. Occurrences: 181. Trajectory: [137., 57.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'L ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'On']
VE  : ['On', 'On', 'On']
44. Occurrences: 167. Trajectory: [137.,129.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
45. Occurrences: 167. Trajectory: [145.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
46. Occurrences: 166. Trajectory: [137.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
47. Occurrences: 165. Trajectory: [137.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
48. Occurrences: 163. Trajectory: [137.,217.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'On']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
49. Occurrences: 163. Trajectory: [137.,129., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
50. Occurrences: 160. Trajectory: [57.,67.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
'''

### State 136 Without Diabetes

In [ ]:
'''
24696
1. Occurrences: 8565. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
2. Occurrences: 2574. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
3. Occurrences: 1597. Trajectory: [379.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
4. Occurrences: 897. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,459.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
5. Occurrences: 897. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,459.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
6. Occurrences: 857. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,459.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
7. Occurrences: 792. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,459.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
8. Occurrences: 762. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,459.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
9. Occurrences: 745. Trajectory: [139.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
10. Occurrences: 707. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 459.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
11. Occurrences: 650. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
12. Occurrences: 606. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
13. Occurrences: 583. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
14. Occurrences: 573. Trajectory: [459.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
15. Occurrences: 558. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
16. Occurrences: 537. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
17. Occurrences: 520. Trajectory: [219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
18. Occurrences: 501. Trajectory: [219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
19. Occurrences: 479. Trajectory: [219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
20. Occurrences: 479. Trajectory: [219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
21. Occurrences: 466. Trajectory: [459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
22. Occurrences: 454. Trajectory: [219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
23. Occurrences: 437. Trajectory: [219.,459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
24. Occurrences: 434. Trajectory: [139.,379.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
25. Occurrences: 414. Trajectory: [219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
26. Occurrences: 390. Trajectory: [219.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
27. Occurrences: 388. Trajectory: [459.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
28. Occurrences: 382. Trajectory: [219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
29. Occurrences: 380. Trajectory: [219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
30. Occurrences: 377. Trajectory: [219.,219.,459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
31. Occurrences: 371. Trajectory: [219.,219.,219.,459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
32. Occurrences: 368. Trajectory: [219.,459.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
33. Occurrences: 357. Trajectory: [219.,219.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
34. Occurrences: 356. Trajectory: [219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
35. Occurrences: 339. Trajectory: [219.,219.,459.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
36. Occurrences: 327. Trajectory: [459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
37. Occurrences: 316. Trajectory: [219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
38. Occurrences: 313. Trajectory: [219.,219.,219.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
39. Occurrences: 313. Trajectory: [459.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
40. Occurrences: 303. Trajectory: [219.,219.,219.,459.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
41. Occurrences: 299. Trajectory: [219.,219.,219.,219.,459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
42. Occurrences: 291. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,459.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
43. Occurrences: 286. Trajectory: [219.,459.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
44. Occurrences: 281. Trajectory: [219.,219.,219.,219.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
45. Occurrences: 278. Trajectory: [459.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
46. Occurrences: 277. Trajectory: [219.,219.,459.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
47. Occurrences: 273. Trajectory: [219.,219.,219.,219.,219.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
48. Occurrences: 269. Trajectory: [219.,459.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
49. Occurrences: 259. Trajectory: [219.,219.,219.,459.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
50. Occurrences: 258. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,459.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
'''

### State 143, Diabetes

In [ ]:
'''
48090
1. Occurrences: 2831. Trajectory: [225.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 2760. Trajectory: [209.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 1394. Trajectory: [65.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
4. Occurrences: 1365. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
5. Occurrences: 944. Trajectory: [137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
6. Occurrences: 934. Trajectory: [129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
7. Occurrences: 931. Trajectory: [145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
8. Occurrences: 930. Trajectory: [129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
9. Occurrences: 926. Trajectory: [145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
10. Occurrences: 919. Trajectory: [137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
11. Occurrences: 907. Trajectory: [137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
12. Occurrences: 907. Trajectory: [137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
13. Occurrences: 724. Trajectory: [129.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'VL']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
14. Occurrences: 695. Trajectory: [129., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['L ', 'VL']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
15. Occurrences: 692. Trajectory: [145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
16. Occurrences: 673. Trajectory: [145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
17. Occurrences: 622. Trajectory: [217.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
18. Occurrences: 541. Trajectory: [217.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
19. Occurrences: 364. Trajectory: [145.,153., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
20. Occurrences: 357. Trajectory: [129.,121.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
21. Occurrences: 355. Trajectory: [129.,121., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
22. Occurrences: 346. Trajectory: [145.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
23. Occurrences: 296. Trajectory: [ 57.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
24. Occurrences: 289. Trajectory: [145.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
25. Occurrences: 280. Trajectory: [137.,137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
26. Occurrences: 268. Trajectory: [129.,129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
27. Occurrences: 266. Trajectory: [137.,137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
28. Occurrences: 265. Trajectory: [137.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
29. Occurrences: 261. Trajectory: [129.,129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
30. Occurrences: 257. Trajectory: [137.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
31. Occurrences: 256. Trajectory: [137.,129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
32. Occurrences: 246. Trajectory: [145.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
33. Occurrences: 245. Trajectory: [137.,137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
34. Occurrences: 241. Trajectory: [137.,129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
35. Occurrences: 241. Trajectory: [137.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
36. Occurrences: 225. Trajectory: [137.,129., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
37. Occurrences: 224. Trajectory: [137., 57.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'L ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'On']
VE  : ['On', 'On', 'On']
38. Occurrences: 218. Trajectory: [129.,129., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
39. Occurrences: 216. Trajectory: [145.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
40. Occurrences: 213. Trajectory: [145.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
41. Occurrences: 206. Trajectory: [137.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
42. Occurrences: 205. Trajectory: [137.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
43. Occurrences: 202. Trajectory: [129.,129.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
44. Occurrences: 201. Trajectory: [137.,129.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
45. Occurrences: 193. Trajectory: [129.,121.,121., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL', 'VL']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
46. Occurrences: 189. Trajectory: [145.,153.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
47. Occurrences: 184. Trajectory: [137.,217.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'On']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
48. Occurrences: 183. Trajectory: [137.,217.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'On']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
49. Occurrences: 157. Trajectory: [129.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
50. Occurrences: 156. Trajectory: [129.,121.,121.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL', 'VL']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
'''

### State 63, Diabetes

In [ ]:
'''
39594
1. Occurrences: 20039. Trajectory: [227.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
2. Occurrences: 4999. Trajectory: [67.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
3. Occurrences: 2887. Trajectory: [219.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
4. Occurrences: 2855. Trajectory: [219.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
5. Occurrences: 874. Trajectory: [ 59.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
6. Occurrences: 582. Trajectory: [219.,221.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'On', 'On']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
7. Occurrences: 553. Trajectory: [219.,221.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'On', 'On']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
8. Occurrences: 445. Trajectory: [147., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
9. Occurrences: 418. Trajectory: [147.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
10. Occurrences: 379. Trajectory: [147.,153., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
11. Occurrences: 367. Trajectory: [147., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
12. Occurrences: 359. Trajectory: [139., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
13. Occurrences: 314. Trajectory: [147.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
14. Occurrences: 297. Trajectory: [147.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
15. Occurrences: 297. Trajectory: [139., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
16. Occurrences: 268. Trajectory: [139.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
17. Occurrences: 255. Trajectory: [139.,129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
18. Occurrences: 242. Trajectory: [139.,129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
19. Occurrences: 241. Trajectory: [139.,137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
20. Occurrences: 239. Trajectory: [59.,67.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
21. Occurrences: 237. Trajectory: [139.,137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
22. Occurrences: 231. Trajectory: [139.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
23. Occurrences: 230. Trajectory: [147.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
24. Occurrences: 229. Trajectory: [139.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
25. Occurrences: 228. Trajectory: [139.,137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
26. Occurrences: 223. Trajectory: [147.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
27. Occurrences: 221. Trajectory: [147.,153.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
28. Occurrences: 203. Trajectory: [147.,153.,153., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
29. Occurrences: 199. Trajectory: [139.,129., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
30. Occurrences: 190. Trajectory: [147.,137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
31. Occurrences: 182. Trajectory: [139.,129.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
32. Occurrences: 179. Trajectory: [139.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
33. Occurrences: 173. Trajectory: [147.,153.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
34. Occurrences: 171. Trajectory: [147.,137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
35. Occurrences: 169. Trajectory: [147.,153., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
36. Occurrences: 167. Trajectory: [219., 53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
37. Occurrences: 162. Trajectory: [147.,137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
38. Occurrences: 160. Trajectory: [219., 69.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
39. Occurrences: 154. Trajectory: [147.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
40. Occurrences: 150. Trajectory: [139.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
41. Occurrences: 127. Trajectory: [219.,133.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
42. Occurrences: 125. Trajectory: [ 59.,219.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['L ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'On']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'On']
43. Occurrences: 124. Trajectory: [147.,145.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
44. Occurrences: 120. Trajectory: [ 59.,219.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['L ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'On']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'On']
45. Occurrences: 119. Trajectory: [139.,377.,381.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
46. Occurrences: 119. Trajectory: [219.,141.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
47. Occurrences: 117. Trajectory: [219.,149.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
48. Occurrences: 116. Trajectory: [219.,141.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
49. Occurrences: 112. Trajectory: [219.,221.,221.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'H ']
AB  : ['Of', 'On', 'On', 'On']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
50. Occurrences: 112. Trajectory: [147.,385.,381.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
'''

### State 56, Non Diabetes

In [ ]:
'''
28189
1. Occurrences: 5987. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
2. Occurrences: 3641. Trajectory: [379.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
3. Occurrences: 1812. Trajectory: [ 59.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['L ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
4. Occurrences: 1720. Trajectory: [139.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
5. Occurrences: 1151. Trajectory: [299.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
6. Occurrences: 972. Trajectory: [ 59.,379.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ']
SBPs: ['L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
7. Occurrences: 938. Trajectory: [139.,379.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
8. Occurrences: 644. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,459.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
9. Occurrences: 637. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,459.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
10. Occurrences: 624. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,459.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
11. Occurrences: 580. Trajectory: [ 59.,139.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['L ', 'N ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
12. Occurrences: 542. Trajectory: [139.,139.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
13. Occurrences: 538. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,459.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
14. Occurrences: 534. Trajectory: [ 59., 59.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['L ', 'L ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
15. Occurrences: 517. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,459.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
16. Occurrences: 483. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
17. Occurrences: 451. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 459.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
18. Occurrences: 421. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
19. Occurrences: 390. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
20. Occurrences: 367. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
21. Occurrences: 362. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
22. Occurrences: 353. Trajectory: [139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
23. Occurrences: 340. Trajectory: [139.,219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
24. Occurrences: 333. Trajectory: [139.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
25. Occurrences: 333. Trajectory: [139.,219.,219.,219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
26. Occurrences: 318. Trajectory: [139.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
27. Occurrences: 309. Trajectory: [139.,219.,219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
28. Occurrences: 303. Trajectory: [ 59.,299.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ']
SBPs: ['L ', 'L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
29. Occurrences: 297. Trajectory: [139.,219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
30. Occurrences: 293. Trajectory: [139.,459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
31. Occurrences: 282. Trajectory: [139.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
32. Occurrences: 279. Trajectory: [299.,303.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['L ', 'L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
33. Occurrences: 272. Trajectory: [ 59., 59.,379.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ']
SBPs: ['L ', 'L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
34. Occurrences: 271. Trajectory: [139.,219.,219.,459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
35. Occurrences: 269. Trajectory: [139.,219.,459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
36. Occurrences: 263. Trajectory: [ 59.,139.,379.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ']
SBPs: ['L ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
37. Occurrences: 261. Trajectory: [139.,459.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
38. Occurrences: 260. Trajectory: [139.,219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
39. Occurrences: 260. Trajectory: [139.,219.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
40. Occurrences: 249. Trajectory: [139.,219.,459.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
41. Occurrences: 248. Trajectory: [139.,219.,219.,219.,459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
42. Occurrences: 247. Trajectory: [139.,219.,219.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
43. Occurrences: 225. Trajectory: [139.,139.,379.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
44. Occurrences: 223. Trajectory: [139.,219.,219.,459.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
45. Occurrences: 220. Trajectory: [139.,219.,219.,219.,459.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
46. Occurrences: 216. Trajectory: [139.,459.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
47. Occurrences: 213. Trajectory: [139.,459.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
48. Occurrences: 201. Trajectory: [139.,219.,219.,219.,219.,219.,459.,383.,376.] Num abnormal of last state: 0
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
49. Occurrences: 195. Trajectory: [ 59.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,459.,463.,463.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ', 'N ', 'N ']
SBPs: ['L ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
50. Occurrences: 188. Trajectory: [139.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,459.] Num abnormal of last state: 1
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'N ']
SBPs: ['N ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
'''

### State 56, Diabetes

In [ ]:
'''
43269
1. Occurrences: 18105. Trajectory: [227.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
2. Occurrences: 4473. Trajectory: [67.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
3. Occurrences: 2579. Trajectory: [219.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
4. Occurrences: 2556. Trajectory: [219.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
5. Occurrences: 804. Trajectory: [ 59.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
6. Occurrences: 499. Trajectory: [219.,221.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'On', 'On']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
7. Occurrences: 497. Trajectory: [219.,221.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'On', 'On']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
8. Occurrences: 396. Trajectory: [147., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
9. Occurrences: 393. Trajectory: [147.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
10. Occurrences: 325. Trajectory: [147.,153., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
11. Occurrences: 317. Trajectory: [147., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
12. Occurrences: 308. Trajectory: [139., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
13. Occurrences: 298. Trajectory: [139., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
14. Occurrences: 284. Trajectory: [379.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
15. Occurrences: 283. Trajectory: [147.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
16. Occurrences: 273. Trajectory: [147.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
17. Occurrences: 229. Trajectory: [139.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
18. Occurrences: 226. Trajectory: [139.,129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
19. Occurrences: 223. Trajectory: [139.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
20. Occurrences: 221. Trajectory: [139.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
21. Occurrences: 220. Trajectory: [139.,137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
22. Occurrences: 208. Trajectory: [387.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
23. Occurrences: 206. Trajectory: [59.,67.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
24. Occurrences: 203. Trajectory: [147.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
25. Occurrences: 197. Trajectory: [139.,129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
26. Occurrences: 196. Trajectory: [139.,137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
27. Occurrences: 194. Trajectory: [139.,137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
28. Occurrences: 186. Trajectory: [387.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
29. Occurrences: 183. Trajectory: [147.,153.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
30. Occurrences: 181. Trajectory: [147.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
31. Occurrences: 170. Trajectory: [147.,153., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
32. Occurrences: 170. Trajectory: [379.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
33. Occurrences: 165. Trajectory: [139.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
34. Occurrences: 162. Trajectory: [139.,129.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
35. Occurrences: 157. Trajectory: [147.,153.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
36. Occurrences: 156. Trajectory: [147.,137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
37. Occurrences: 155. Trajectory: [147.,137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
38. Occurrences: 155. Trajectory: [147.,153.,153., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
39. Occurrences: 149. Trajectory: [139.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
40. Occurrences: 147. Trajectory: [147.,137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
41. Occurrences: 147. Trajectory: [139.,129., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
42. Occurrences: 139. Trajectory: [147.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
43. Occurrences: 134. Trajectory: [379.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
44. Occurrences: 130. Trajectory: [ 59.,219.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['L ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'On']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'On']
45. Occurrences: 127. Trajectory: [219.,141.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
46. Occurrences: 123. Trajectory: [219., 53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
47. Occurrences: 123. Trajectory: [219., 69.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
48. Occurrences: 119. Trajectory: [219.,149.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
49. Occurrences: 115. Trajectory: [459.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
50. Occurrences: 111. Trajectory: [ 59.,219.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['L ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'On']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'On']
'''

### State 377, Diabetes

In [ ]:
'''
49255
1. Occurrences: 11712. Trajectory: [381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
2. Occurrences: 6671. Trajectory: [373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['L ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
3. Occurrences: 6395. Trajectory: [389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['H ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
4. Occurrences: 785. Trajectory: [373.,368.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
5. Occurrences: 779. Trajectory: [381.,368.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
6. Occurrences: 776. Trajectory: [381.,368.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
7. Occurrences: 771. Trajectory: [373.,368.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
8. Occurrences: 768. Trajectory: [381.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
9. Occurrences: 765. Trajectory: [389.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
10. Occurrences: 746. Trajectory: [381.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
11. Occurrences: 700. Trajectory: [389.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
12. Occurrences: 448. Trajectory: [373.,360.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
13. Occurrences: 437. Trajectory: [389.,392.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
14. Occurrences: 298. Trajectory: [389.,392.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
15. Occurrences: 287. Trajectory: [373.,360.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
16. Occurrences: 232. Trajectory: [389.,392.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'Of']
17. Occurrences: 191. Trajectory: [381.,384.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
18. Occurrences: 190. Trajectory: [373.,360.,365.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'Of']
19. Occurrences: 183. Trajectory: [381.,368.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
20. Occurrences: 178. Trajectory: [389.,392.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'On', 'Of']
21. Occurrences: 169. Trajectory: [389.,384.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
22. Occurrences: 167. Trajectory: [373.,368.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
23. Occurrences: 139. Trajectory: [373.,360.,365.,365.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL', 'VL', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'On', 'Of']
24. Occurrences: 136. Trajectory: [389.,384.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'Of']
25. Occurrences: 126. Trajectory: [389.,704.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
26. Occurrences: 116. Trajectory: [373.,368.,365.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'VL', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'Of']
27. Occurrences: 115. Trajectory: [381.,688.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
28. Occurrences: 115. Trajectory: [389.,584.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'N ']
POs : ['N ', 'L ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
29. Occurrences: 113. Trajectory: [381.,408.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
30. Occurrences: 113. Trajectory: [381.,336.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'L ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
31. Occurrences: 113. Trajectory: [381.,384.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'Of']
32. Occurrences: 111. Trajectory: [381.,568.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'N ']
POs : ['N ', 'L ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
33. Occurrences: 110. Trajectory: [373.,688.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
34. Occurrences: 109. Trajectory: [381.,368.,373.,368.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ', 'L ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
35. Occurrences: 108. Trajectory: [381.,368.,365.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'Of']
36. Occurrences: 108. Trajectory: [389.,424.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
37. Occurrences: 108. Trajectory: [389.,392.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'On', 'On', 'Of']
38. Occurrences: 108. Trajectory: [373.,568.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'N ']
POs : ['N ', 'L ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
39. Occurrences: 105. Trajectory: [381.,704.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
40. Occurrences: 103. Trajectory: [381.,584.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'N ']
POs : ['N ', 'L ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
41. Occurrences: 99. Trajectory: [381.,424.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
42. Occurrences: 97. Trajectory: [373.,408.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
43. Occurrences: 96. Trajectory: [373.,360.,365.,365.,365.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL', 'VL', 'VL', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'On', 'On', 'Of']
44. Occurrences: 96. Trajectory: [373.,368.,373.,368.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'L ', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
45. Occurrences: 93. Trajectory: [389.,384.,389.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'H ', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
46. Occurrences: 93. Trajectory: [373.,560.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'N ']
POs : ['N ', 'L ']
GL  : ['L ', 'VL']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
47. Occurrences: 91. Trajectory: [373.,680.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'VL']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
48. Occurrences: 90. Trajectory: [381.,368.,373.,368.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ', 'L ', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
49. Occurrences: 90. Trajectory: [373.,368.,373.,368.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'L ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of', 'On', 'Of']
50. Occurrences: 88. Trajectory: [389.,384.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'On', 'On', 'Of']
'''

### State 377, No Diabetes

In [ ]:
'''
28135
1. Occurrences: 45948. Trajectory: [376.] Num abnormal of last state: 0
HRs : ['N ']
SBPs: ['N ']
POs : ['N ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
2. Occurrences: 2126. Trajectory: [296.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of']
3. Occurrences: 640. Trajectory: [296.,303.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['L ', 'L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'Of']
4. Occurrences: 575. Trajectory: [336.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of']
5. Occurrences: 540. Trajectory: [136.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
6. Occurrences: 481. Trajectory: [616.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of']
7. Occurrences: 448. Trajectory: [456.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of']
8. Occurrences: 393. Trajectory: [456.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'Of']
9. Occurrences: 321. Trajectory: [456.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'Of']
10. Occurrences: 285. Trajectory: [456.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'Of']
11. Occurrences: 257. Trajectory: [456.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
12. Occurrences: 251. Trajectory: [456.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
13. Occurrences: 212. Trajectory: [288.,293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,
 293.,293.,293.,293.,293.,293.] Num abnormal of last state: 2
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
14. Occurrences: 209. Trajectory: [368.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,
 372.,372.,372.,372.,372.,372.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
15. Occurrences: 207. Trajectory: [296.,303.,303.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['L ', 'L ', 'L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'Of']
16. Occurrences: 206. Trajectory: [336.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'Of']
17. Occurrences: 200. Trajectory: [128.,133.,133.,133.,133.,133.,133.,133.,133.,133.,133.,133.,133.,133.,
 133.,133.,133.,133.,133.,133.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
18. Occurrences: 191. Trajectory: [456.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
19. Occurrences: 182. Trajectory: [688.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
20. Occurrences: 180. Trajectory: [408.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
21. Occurrences: 173. Trajectory: [256.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['L ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of']
22. Occurrences: 173. Trajectory: [336.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'Of']
23. Occurrences: 171. Trajectory: [48.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
24. Occurrences: 171. Trajectory: [528.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
25. Occurrences: 170. Trajectory: [456.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
26. Occurrences: 168. Trajectory: [136.,139.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
27. Occurrences: 167. Trajectory: [208.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
28. Occurrences: 158. Trajectory: [248.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
29. Occurrences: 156. Trajectory: [88.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
30. Occurrences: 150. Trajectory: [336.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'Of']
31. Occurrences: 147. Trajectory: [616.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'Of']
32. Occurrences: 141. Trajectory: [568.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
33. Occurrences: 137. Trajectory: [616.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'Of']
34. Occurrences: 136. Trajectory: [456.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
35. Occurrences: 135. Trajectory: [536.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of']
36. Occurrences: 133. Trajectory: [456.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
37. Occurrences: 130. Trajectory: [ 96.,179.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
38. Occurrences: 116. Trajectory: [336.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
39. Occurrences: 115. Trajectory: [216.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,219.,
 219.,219.,219.,219.,219.,219.] Num abnormal of last state: 2
HRs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
40. Occurrences: 112. Trajectory: [456.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
41. Occurrences: 109. Trajectory: [616.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'Of']
42. Occurrences: 109. Trajectory: [336.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
43. Occurrences: 108. Trajectory: [616.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
44. Occurrences: 106. Trajectory: [336.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
45. Occurrences: 98. Trajectory: [136.,379.,376.] Num abnormal of last state: 0
HRs : ['L ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of']
46. Occurrences: 94. Trajectory: [456.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
47. Occurrences: 92. Trajectory: [296.,383.,296.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['L ', 'N ', 'L ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'On', 'Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of', 'On', 'Of']
48. Occurrences: 92. Trajectory: [336.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
49. Occurrences: 88. Trajectory: [176.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
50. Occurrences: 83. Trajectory: [616.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['Of', 'On', 'On', 'On', 'On', 'On', 'Of']
'''

### State 696, Diabetes

In [ ]:
'''
54708
1. Occurrences: 7515. Trajectory: [693.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 7399. Trajectory: [709.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 2970. Trajectory: [381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
4. Occurrences: 1644. Trajectory: [373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['L ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
5. Occurrences: 1638. Trajectory: [389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['H ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
6. Occurrences: 753. Trajectory: [701.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
7. Occurrences: 741. Trajectory: [701.,709.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
8. Occurrences: 598. Trajectory: [461.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
9. Occurrences: 590. Trajectory: [621.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
10. Occurrences: 350. Trajectory: [629.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
11. Occurrences: 344. Trajectory: [453.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
12. Occurrences: 343. Trajectory: [621.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
13. Occurrences: 336. Trajectory: [453.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
14. Occurrences: 327. Trajectory: [469.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
15. Occurrences: 323. Trajectory: [621.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
16. Occurrences: 323. Trajectory: [629.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
17. Occurrences: 322. Trajectory: [469.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
18. Occurrences: 318. Trajectory: [461.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
19. Occurrences: 315. Trajectory: [613.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
20. Occurrences: 310. Trajectory: [461.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
21. Occurrences: 295. Trajectory: [613.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
22. Occurrences: 287. Trajectory: [701.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
23. Occurrences: 210. Trajectory: [389.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
24. Occurrences: 206. Trajectory: [381.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
25. Occurrences: 196. Trajectory: [373.,368.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
26. Occurrences: 190. Trajectory: [373.,368.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
27. Occurrences: 189. Trajectory: [381.,368.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
28. Occurrences: 181. Trajectory: [701.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
29. Occurrences: 179. Trajectory: [701.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
30. Occurrences: 175. Trajectory: [381.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
31. Occurrences: 174. Trajectory: [389.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
32. Occurrences: 174. Trajectory: [381.,368.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
33. Occurrences: 128. Trajectory: [461.,461.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
34. Occurrences: 113. Trajectory: [621.,621.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
35. Occurrences: 105. Trajectory: [373.,360.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
36. Occurrences: 99. Trajectory: [389.,392.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
37. Occurrences: 80. Trajectory: [453.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'L ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
38. Occurrences: 78. Trajectory: [461.,453.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
39. Occurrences: 78. Trajectory: [629.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
40. Occurrences: 76. Trajectory: [621.,613.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
41. Occurrences: 72. Trajectory: [469.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
42. Occurrences: 72. Trajectory: [701.,701.,709.] Num abnormal of last state: 3
HRs : ['H ', 'H ', 'H ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['On', 'On', 'On']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
43. Occurrences: 72. Trajectory: [453.,461.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
44. Occurrences: 72. Trajectory: [701.,701.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ', 'H ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['On', 'On', 'On']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
45. Occurrences: 72. Trajectory: [461.,461.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
46. Occurrences: 71. Trajectory: [461.,469.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
47. Occurrences: 71. Trajectory: [461.,461.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
48. Occurrences: 70. Trajectory: [469.,469.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
49. Occurrences: 70. Trajectory: [373.,360.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
50. Occurrences: 70. Trajectory: [621.,629.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
'''

### State 696, No Diabetes

In [ ]:
'''
32080
1. Occurrences: 4012. Trajectory: [383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
2. Occurrences: 3404. Trajectory: [463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
3. Occurrences: 2901. Trajectory: [463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
4. Occurrences: 2477. Trajectory: [463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
5. Occurrences: 2066. Trajectory: [463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
6. Occurrences: 1878. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
7. Occurrences: 1768. Trajectory: [463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
8. Occurrences: 1740. Trajectory: [703.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
9. Occurrences: 1567. Trajectory: [463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
10. Occurrences: 1439. Trajectory: [703.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
11. Occurrences: 1269. Trajectory: [463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
12. Occurrences: 1186. Trajectory: [703.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
13. Occurrences: 1094. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
14. Occurrences: 1036. Trajectory: [703.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
15. Occurrences: 969. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
16. Occurrences: 916. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
17. Occurrences: 891. Trajectory: [703.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
18. Occurrences: 803. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
19. Occurrences: 783. Trajectory: [703.,703.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
20. Occurrences: 774. Trajectory: [703.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
21. Occurrences: 633. Trajectory: [703.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
22. Occurrences: 632. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
23. Occurrences: 630. Trajectory: [623.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
24. Occurrences: 607. Trajectory: [703.,703.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
25. Occurrences: 557. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
26. Occurrences: 554. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
27. Occurrences: 525. Trajectory: [703.,703.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
28. Occurrences: 486. Trajectory: [703.,703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
29. Occurrences: 476. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,
 376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
30. Occurrences: 436. Trajectory: [703.,703.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
31. Occurrences: 435. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
32. Occurrences: 411. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
33. Occurrences: 390. Trajectory: [703.,703.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
34. Occurrences: 378. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
35. Occurrences: 361. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
36. Occurrences: 321. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,383.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
37. Occurrences: 320. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
38. Occurrences: 315. Trajectory: [703.,703.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
39. Occurrences: 309. Trajectory: [703.,703.,703.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'H ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
40. Occurrences: 286. Trajectory: [703.,703.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
41. Occurrences: 285. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
42. Occurrences: 278. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
43. Occurrences: 264. Trajectory: [703.,703.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
44. Occurrences: 263. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
45. Occurrences: 249. Trajectory: [703.,623.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
46. Occurrences: 247. Trajectory: [703.,703.,703.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'H ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
47. Occurrences: 228. Trajectory: [703.,703.,703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['H ', 'H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
48. Occurrences: 227. Trajectory: [703.,703.,703.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'H ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
49. Occurrences: 220. Trajectory: [623.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
50. Occurrences: 218. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
'''

### State 703, Diabetes

In [ ]:
'''
54877
1. Occurrences: 7193. Trajectory: [693.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 7122. Trajectory: [709.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 2943. Trajectory: [381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
4. Occurrences: 1680. Trajectory: [373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['L ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
5. Occurrences: 1604. Trajectory: [389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['H ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
6. Occurrences: 724. Trajectory: [701.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
7. Occurrences: 723. Trajectory: [701.,709.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
8. Occurrences: 608. Trajectory: [621.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
9. Occurrences: 575. Trajectory: [461.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
10. Occurrences: 382. Trajectory: [533.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
11. Occurrences: 368. Trajectory: [549.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
12. Occurrences: 350. Trajectory: [621.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
13. Occurrences: 350. Trajectory: [629.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
14. Occurrences: 340. Trajectory: [613.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
15. Occurrences: 327. Trajectory: [453.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
16. Occurrences: 326. Trajectory: [461.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
17. Occurrences: 320. Trajectory: [629.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
18. Occurrences: 310. Trajectory: [621.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
19. Occurrences: 303. Trajectory: [453.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
20. Occurrences: 302. Trajectory: [461.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
21. Occurrences: 301. Trajectory: [469.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
22. Occurrences: 294. Trajectory: [469.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
23. Occurrences: 288. Trajectory: [613.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
24. Occurrences: 276. Trajectory: [701.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
25. Occurrences: 234. Trajectory: [381.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
26. Occurrences: 205. Trajectory: [389.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
27. Occurrences: 204. Trajectory: [381.,368.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
28. Occurrences: 188. Trajectory: [373.,368.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
29. Occurrences: 188. Trajectory: [381.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
30. Occurrences: 179. Trajectory: [381.,368.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
31. Occurrences: 176. Trajectory: [373.,368.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
32. Occurrences: 172. Trajectory: [701.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
33. Occurrences: 165. Trajectory: [389.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
34. Occurrences: 157. Trajectory: [701.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
35. Occurrences: 113. Trajectory: [621.,621.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
36. Occurrences: 111. Trajectory: [373.,360.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'L ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
37. Occurrences: 102. Trajectory: [389.,392.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
38. Occurrences: 97. Trajectory: [461.,461.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
39. Occurrences: 89. Trajectory: [389.,392.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
40. Occurrences: 85. Trajectory: [629.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
41. Occurrences: 78. Trajectory: [701.,701.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ', 'H ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['On', 'On', 'On']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
42. Occurrences: 78. Trajectory: [621.,621.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
43. Occurrences: 78. Trajectory: [613.,621.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
44. Occurrences: 77. Trajectory: [621.,629.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
45. Occurrences: 76. Trajectory: [453.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'L ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
46. Occurrences: 74. Trajectory: [701.,701.,709.] Num abnormal of last state: 3
HRs : ['H ', 'H ', 'H ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['On', 'On', 'On']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'On']
47. Occurrences: 74. Trajectory: [613.,365.,373.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'VL', 'L ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
48. Occurrences: 72. Trajectory: [469.,469.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
49. Occurrences: 72. Trajectory: [629.,621.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
50. Occurrences: 71. Trajectory: [629.,629.,381.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
'''

### State 703, No Diabetes

In [ ]:
'''
32080
1. Occurrences: 4012. Trajectory: [383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
2. Occurrences: 3404. Trajectory: [463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
3. Occurrences: 2901. Trajectory: [463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
4. Occurrences: 2477. Trajectory: [463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
5. Occurrences: 2066. Trajectory: [463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
6. Occurrences: 1878. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
7. Occurrences: 1768. Trajectory: [463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
8. Occurrences: 1740. Trajectory: [703.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
9. Occurrences: 1567. Trajectory: [463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
10. Occurrences: 1439. Trajectory: [703.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
11. Occurrences: 1269. Trajectory: [463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
12. Occurrences: 1186. Trajectory: [703.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
13. Occurrences: 1094. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
14. Occurrences: 1036. Trajectory: [703.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
15. Occurrences: 969. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
16. Occurrences: 916. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
17. Occurrences: 891. Trajectory: [703.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
18. Occurrences: 803. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
19. Occurrences: 783. Trajectory: [703.,703.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
20. Occurrences: 774. Trajectory: [703.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
21. Occurrences: 633. Trajectory: [703.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
22. Occurrences: 632. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
23. Occurrences: 630. Trajectory: [623.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'Of']
24. Occurrences: 607. Trajectory: [703.,703.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
25. Occurrences: 557. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
26. Occurrences: 554. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
27. Occurrences: 525. Trajectory: [703.,703.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
28. Occurrences: 486. Trajectory: [703.,703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
29. Occurrences: 476. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,
 376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
30. Occurrences: 436. Trajectory: [703.,703.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
31. Occurrences: 435. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
32. Occurrences: 411. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
33. Occurrences: 390. Trajectory: [703.,703.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
34. Occurrences: 378. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
35. Occurrences: 361. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
36. Occurrences: 321. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,383.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
37. Occurrences: 320. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
38. Occurrences: 315. Trajectory: [703.,703.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
39. Occurrences: 309. Trajectory: [703.,703.,703.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'H ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
40. Occurrences: 286. Trajectory: [703.,703.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
41. Occurrences: 285. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
42. Occurrences: 278. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
43. Occurrences: 264. Trajectory: [703.,703.,463.,463.,463.,463.,463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
44. Occurrences: 263. Trajectory: [463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
45. Occurrences: 249. Trajectory: [703.,623.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
46. Occurrences: 247. Trajectory: [703.,703.,703.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'H ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
47. Occurrences: 228. Trajectory: [703.,703.,703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 463.,463.,463.,463.,463.,463.] Num abnormal of last state: 1
HRs : ['H ', 'H ', 'H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
48. Occurrences: 227. Trajectory: [703.,703.,703.,463.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'H ', 'H ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
49. Occurrences: 220. Trajectory: [623.,463.,383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'Of']
50. Occurrences: 218. Trajectory: [703.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,463.,
 383.,376.] Num abnormal of last state: 0
HRs : ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
'''

### State 439, Diabetes

In [ ]:
'''
54405
1. Occurrences: 10028. Trajectory: [437.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 4215. Trajectory: [429.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 2244. Trajectory: [389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['H ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
4. Occurrences: 1503. Trajectory: [397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
5. Occurrences: 1143. Trajectory: [397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
6. Occurrences: 800. Trajectory: [397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
7. Occurrences: 794. Trajectory: [477.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
8. Occurrences: 568. Trajectory: [277.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
9. Occurrences: 551. Trajectory: [477.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
10. Occurrences: 528. Trajectory: [397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
11. Occurrences: 481. Trajectory: [357.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ']
GL  : ['VH', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
12. Occurrences: 462. Trajectory: [469.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
13. Occurrences: 414. Trajectory: [469.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
14. Occurrences: 394. Trajectory: [397.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
15. Occurrences: 366. Trajectory: [477.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
16. Occurrences: 338. Trajectory: [357.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
17. Occurrences: 289. Trajectory: [397.,397.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
18. Occurrences: 282. Trajectory: [349.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
19. Occurrences: 274. Trajectory: [477.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
20. Occurrences: 257. Trajectory: [389.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
21. Occurrences: 242. Trajectory: [349.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
22. Occurrences: 238. Trajectory: [477.,477.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
23. Occurrences: 232. Trajectory: [269.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
24. Occurrences: 227. Trajectory: [357.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
25. Occurrences: 225. Trajectory: [389.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
26. Occurrences: 197. Trajectory: [477.,477.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of']
27. Occurrences: 194. Trajectory: [397.,389.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'On', 'Of']
28. Occurrences: 194. Trajectory: [397.,397.,397.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
29. Occurrences: 182. Trajectory: [477.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
30. Occurrences: 179. Trajectory: [397.,389.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'On', 'Of']
31. Occurrences: 173. Trajectory: [477.,469.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
32. Occurrences: 167. Trajectory: [357.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
33. Occurrences: 149. Trajectory: [389.,392.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'Of']
34. Occurrences: 144. Trajectory: [477.,469.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
35. Occurrences: 134. Trajectory: [397.,397.,389.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of', 'On', 'Of']
36. Occurrences: 130. Trajectory: [397.,397.,389.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of', 'On', 'Of']
37. Occurrences: 125. Trajectory: [477.,477.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'Of']
38. Occurrences: 124. Trajectory: [477.,397.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
39. Occurrences: 123. Trajectory: [397.,397.,397.,397.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
40. Occurrences: 111. Trajectory: [389.,392.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'On', 'On', 'Of']
41. Occurrences: 108. Trajectory: [397.,389.,392.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'On', 'Of']
42. Occurrences: 105. Trajectory: [397.,397.,397.,397.,397.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
43. Occurrences: 102. Trajectory: [469.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
44. Occurrences: 101. Trajectory: [397.,397.,397.,389.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'Of', 'On', 'Of']
45. Occurrences: 101. Trajectory: [357.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
46. Occurrences: 99. Trajectory: [477.,389.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'On', 'Of']
47. Occurrences: 93. Trajectory: [357.,357.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['L ', 'L ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
48. Occurrences: 92. Trajectory: [477.,397.,397.,397.,397.,397.,397.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'Of']
49. Occurrences: 91. Trajectory: [469.,469.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
50. Occurrences: 90. Trajectory: [477.,389.,384.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'On', 'Of']
'''

### State 439 No Diabetes

In [ ]:
'''
72031
1. Occurrences: 10918. Trajectory: [437.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 1345. Trajectory: [429.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 1317. Trajectory: [413.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
4. Occurrences: 1131. Trajectory: [277.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
5. Occurrences: 202. Trajectory: [389.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of']
6. Occurrences: 180. Trajectory: [389.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of']
7. Occurrences: 171. Trajectory: [397.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of']
8. Occurrences: 158. Trajectory: [269.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
9. Occurrences: 144. Trajectory: [253.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
10. Occurrences: 136. Trajectory: [397.,396.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of']
11. Occurrences: 93. Trajectory: [389.,388.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'On', 'Of']
12. Occurrences: 84. Trajectory: [397.,388.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'On', 'Of']
13. Occurrences: 76. Trajectory: [469.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
14. Occurrences: 73. Trajectory: [397.,388.,384.,389.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'On', 'Of', 'Of']
15. Occurrences: 71. Trajectory: [397.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,
 372.,372.,372.,372.,372.,372.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
16. Occurrences: 70. Trajectory: [397.,396.,396.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of']
17. Occurrences: 70. Trajectory: [397.,396.,388.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'On', 'Of']
18. Occurrences: 69. Trajectory: [477.,389.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
19. Occurrences: 68. Trajectory: [477.,389.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
20. Occurrences: 67. Trajectory: [469.,389.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
21. Occurrences: 67. Trajectory: [373.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,
 372.,372.,372.,372.,372.,372.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
22. Occurrences: 65. Trajectory: [397.,396.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,
 372.,372.,372.,372.,372.,372.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
23. Occurrences: 65. Trajectory: [397.,388.,384.,389.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'On', 'Of', 'Of']
24. Occurrences: 65. Trajectory: [349.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
25. Occurrences: 61. Trajectory: [397.,396.,396.,396.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of']
26. Occurrences: 61. Trajectory: [389.,388.,384.,389.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'On', 'Of', 'Of']
27. Occurrences: 61. Trajectory: [477.,397.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
28. Occurrences: 59. Trajectory: [469.,389.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
29. Occurrences: 57. Trajectory: [389.,388.,384.,389.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'On', 'Of', 'Of']
30. Occurrences: 57. Trajectory: [349.,389.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
31. Occurrences: 56. Trajectory: [397.,396.,396.,396.,396.,396.,396.,396.,396.,396.,396.,396.,396.,396.,
 396.,396.,396.,396.,396.,396.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'VH']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
32. Occurrences: 54. Trajectory: [397.,396.,396.,396.,396.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
33. Occurrences: 52. Trajectory: [397.,396.,388.,384.,389.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'On', 'Of', 'Of']
34. Occurrences: 48. Trajectory: [357.,397.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
35. Occurrences: 47. Trajectory: [477.,397.,396.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of', 'Of']
36. Occurrences: 46. Trajectory: [349.,389.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
37. Occurrences: 43. Trajectory: [357.,389.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
38. Occurrences: 43. Trajectory: [357.,389.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ']
POs : ['L ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of']
39. Occurrences: 42. Trajectory: [469.,469.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'Of']
40. Occurrences: 41. Trajectory: [397.,396.,396.,396.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,
 372.,372.,372.,372.,372.,372.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
41. Occurrences: 39. Trajectory: [333.,332.,332.,332.,332.,332.,332.,332.,332.,332.,332.,332.,332.,332.,
 332.,332.,332.,332.,332.,332.] Num abnormal of last state: 2
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
GL  : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
42. Occurrences: 38. Trajectory: [397.,396.,396.,388.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'On', 'Of']
43. Occurrences: 36. Trajectory: [397.,396.,396.,396.,396.,396.,372.,372.,372.,372.,372.,372.,372.,372.,
 372.,372.,372.,372.,372.,372.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'VH', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
44. Occurrences: 36. Trajectory: [293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,293.,
 293.,293.,293.,293.,293.,293.] Num abnormal of last state: 2
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
45. Occurrences: 36. Trajectory: [477.,389.,388.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'H ', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of', 'On', 'Of']
46. Occurrences: 36. Trajectory: [453.,373.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,
 372.,372.,372.,372.,372.,372.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
47. Occurrences: 36. Trajectory: [469.,389.,388.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of', 'On', 'Of']
48. Occurrences: 35. Trajectory: [477.,397.,396.,396.,396.,388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'VH', 'VH', 'H ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of', 'Of', 'Of', 'Of']
49. Occurrences: 34. Trajectory: [397.,396.,396.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,372.,
 372.,372.,372.,372.,372.,372.] Num abnormal of last state: 1
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ', 'L ']
AB  : ['On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On', 'On']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
50. Occurrences: 34. Trajectory: [477.,397.,396.,388.,384.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ', 'N ', 'N ', 'N ', 'N ']
GL  : ['VH', 'VH', 'VH', 'H ', 'H ', 'N ', 'N ']
AB  : ['On', 'On', 'On', 'On', 'Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of', 'Of', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of', 'Of', 'Of', 'On', 'Of']
'''

## Record Data for Random Policy (top 50 Trajectories over 100000 episodes)

### State 136 With Diabetes

In [ ]:
'''
34818
1. Occurrences: 5509. Trajectory: [231.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['On']
2. Occurrences: 5131. Trajectory: [230.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
3. Occurrences: 5079. Trajectory: [227.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
4. Occurrences: 4664. Trajectory: [226.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
5. Occurrences: 574. Trajectory: [190.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
6. Occurrences: 566. Trajectory: [182.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
7. Occurrences: 524. Trajectory: [186.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
8. Occurrences: 502. Trajectory: [178.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
9. Occurrences: 399. Trajectory: [92.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
10. Occurrences: 398. Trajectory: [108.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
11. Occurrences: 365. Trajectory: [65.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
12. Occurrences: 352. Trajectory: [376.] Num abnormal of last state: 0
HRs : ['N ']
SBPs: ['N ']
POs : ['N ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
13. Occurrences: 351. Trajectory: [208.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
14. Occurrences: 346. Trajectory: [223.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
15. Occurrences: 343. Trajectory: [223.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
16. Occurrences: 342. Trajectory: [225.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
17. Occurrences: 335. Trajectory: [222.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
18. Occurrences: 334. Trajectory: [209.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
19. Occurrences: 323. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
20. Occurrences: 318. Trajectory: [222.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
21. Occurrences: 316. Trajectory: [219.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
22. Occurrences: 315. Trajectory: [64.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
23. Occurrences: 312. Trajectory: [48.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
24. Occurrences: 306. Trajectory: [88.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
25. Occurrences: 305. Trajectory: [141.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
26. Occurrences: 305. Trajectory: [223.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
27. Occurrences: 297. Trajectory: [141.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
28. Occurrences: 291. Trajectory: [222.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
29. Occurrences: 291. Trajectory: [223.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
30. Occurrences: 289. Trajectory: [218.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
31. Occurrences: 288. Trajectory: [219.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
32. Occurrences: 285. Trajectory: [224.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
33. Occurrences: 278. Trajectory: [140.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
34. Occurrences: 269. Trajectory: [218.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
35. Occurrences: 260. Trajectory: [219.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
36. Occurrences: 258. Trajectory: [104.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
37. Occurrences: 256. Trajectory: [140.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
38. Occurrences: 255. Trajectory: [219.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
39. Occurrences: 250. Trajectory: [141.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
40. Occurrences: 243. Trajectory: [218.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
41. Occurrences: 240. Trajectory: [149.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
42. Occurrences: 239. Trajectory: [149.,235.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
43. Occurrences: 238. Trajectory: [141.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
44. Occurrences: 230. Trajectory: [222.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
45. Occurrences: 227. Trajectory: [218.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
46. Occurrences: 223. Trajectory: [149.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
47. Occurrences: 218. Trajectory: [140.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
48. Occurrences: 215. Trajectory: [223.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
49. Occurrences: 215. Trajectory: [137.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
50. Occurrences: 207. Trajectory: [133.,215.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
'''

### State 136 Without Diabetes

In [ ]:
'''
77330
1. Occurrences: 864. Trajectory: [182.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
2. Occurrences: 767. Trajectory: [178.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
3. Occurrences: 739. Trajectory: [376.] Num abnormal of last state: 0
HRs : ['N ']
SBPs: ['N ']
POs : ['N ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
4. Occurrences: 245. Trajectory: [133.,211.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
5. Occurrences: 240. Trajectory: [92.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
6. Occurrences: 224. Trajectory: [209.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
7. Occurrences: 217. Trajectory: [133.,210.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
8. Occurrences: 217. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
9. Occurrences: 217. Trajectory: [48.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
10. Occurrences: 215. Trajectory: [132.,211.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
11. Occurrences: 213. Trajectory: [133.,215.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
12. Occurrences: 203. Trajectory: [133.,214.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
13. Occurrences: 193. Trajectory: [208.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
14. Occurrences: 191. Trajectory: [88.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
15. Occurrences: 186. Trajectory: [223.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
16. Occurrences: 183. Trajectory: [132.,215.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
17. Occurrences: 178. Trajectory: [132.,210.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
18. Occurrences: 177. Trajectory: [222.,208.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
19. Occurrences: 174. Trajectory: [222.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
20. Occurrences: 171. Trajectory: [223.,208.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
21. Occurrences: 158. Trajectory: [132.,214.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
22. Occurrences: 148. Trajectory: [129.,215.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
23. Occurrences: 148. Trajectory: [129.,214.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
24. Occurrences: 140. Trajectory: [128.,211.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
25. Occurrences: 133. Trajectory: [219.,208.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
26. Occurrences: 132. Trajectory: [219.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
27. Occurrences: 132. Trajectory: [129.,211.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
28. Occurrences: 130. Trajectory: [218.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
29. Occurrences: 129. Trajectory: [128.,210.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
30. Occurrences: 129. Trajectory: [218.,208.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
31. Occurrences: 120. Trajectory: [129.,210.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
32. Occurrences: 117. Trajectory: [128.,215.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
33. Occurrences: 111. Trajectory: [128.,214.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
34. Occurrences: 111. Trajectory: [223.,178.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
35. Occurrences: 104. Trajectory: [141.,182.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
36. Occurrences: 102. Trajectory: [222.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
37. Occurrences: 99. Trajectory: [223.,182.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
38. Occurrences: 95. Trajectory: [223.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
39. Occurrences: 95. Trajectory: [176.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
40. Occurrences: 93. Trajectory: [223.,212.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
41. Occurrences: 92. Trajectory: [222.,178.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
42. Occurrences: 90. Trajectory: [219.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
43. Occurrences: 90. Trajectory: [141.,178.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
44. Occurrences: 86. Trajectory: [16.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
45. Occurrences: 85. Trajectory: [219.,178.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
46. Occurrences: 85. Trajectory: [140.,182.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
47. Occurrences: 84. Trajectory: [222.,182.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
48. Occurrences: 84. Trajectory: [218.,182.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
49. Occurrences: 81. Trajectory: [219.,212.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
50. Occurrences: 80. Trajectory: [100.,178.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
'''

### State 143, Diabetes

In [ ]:
'''
32119
1. Occurrences: 5615. Trajectory: [231.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['On']
2. Occurrences: 5522. Trajectory: [227.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
3. Occurrences: 5255. Trajectory: [226.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
4. Occurrences: 5093. Trajectory: [230.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
5. Occurrences: 613. Trajectory: [186.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
6. Occurrences: 565. Trajectory: [178.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
7. Occurrences: 561. Trajectory: [182.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
8. Occurrences: 540. Trajectory: [190.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
9. Occurrences: 378. Trajectory: [92.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
10. Occurrences: 373. Trajectory: [209.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
11. Occurrences: 368. Trajectory: [223.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
12. Occurrences: 358. Trajectory: [108.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
13. Occurrences: 346. Trajectory: [225.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
14. Occurrences: 344. Trajectory: [104.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
15. Occurrences: 333. Trajectory: [219.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
16. Occurrences: 321. Trajectory: [222.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
17. Occurrences: 320. Trajectory: [223.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
18. Occurrences: 319. Trajectory: [219.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
19. Occurrences: 316. Trajectory: [218.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
20. Occurrences: 310. Trajectory: [218.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
21. Occurrences: 307. Trajectory: [224.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
22. Occurrences: 299. Trajectory: [222.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
23. Occurrences: 292. Trajectory: [222.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
24. Occurrences: 290. Trajectory: [208.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
25. Occurrences: 282. Trajectory: [219.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
26. Occurrences: 280. Trajectory: [88.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
27. Occurrences: 278. Trajectory: [223.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
28. Occurrences: 277. Trajectory: [223.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
29. Occurrences: 269. Trajectory: [141.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
30. Occurrences: 269. Trajectory: [219.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
31. Occurrences: 263. Trajectory: [222.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
32. Occurrences: 262. Trajectory: [218.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
33. Occurrences: 261. Trajectory: [141.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
34. Occurrences: 260. Trajectory: [141.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
35. Occurrences: 258. Trajectory: [218.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
36. Occurrences: 250. Trajectory: [137.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
37. Occurrences: 247. Trajectory: [140.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
38. Occurrences: 240. Trajectory: [140.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
39. Occurrences: 234. Trajectory: [149.,235.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
40. Occurrences: 232. Trajectory: [140.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
41. Occurrences: 231. Trajectory: [140.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
42. Occurrences: 221. Trajectory: [223.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
43. Occurrences: 220. Trajectory: [137.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
44. Occurrences: 220. Trajectory: [141.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
45. Occurrences: 215. Trajectory: [137.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
46. Occurrences: 213. Trajectory: [223.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
47. Occurrences: 210. Trajectory: [136.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
48. Occurrences: 207. Trajectory: [133.,215.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
49. Occurrences: 207. Trajectory: [149.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
50. Occurrences: 204. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
'''

### State 63, Diabetes

In [ ]:
'''
23941
1. Occurrences: 3762. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 3752. Trajectory: [65.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 3741. Trajectory: [53.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
4. Occurrences: 3740. Trajectory: [69.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
5. Occurrences: 3390. Trajectory: [48.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
6. Occurrences: 3376. Trajectory: [52.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
7. Occurrences: 3357. Trajectory: [64.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
8. Occurrences: 3308. Trajectory: [68.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
9. Occurrences: 2457. Trajectory: [231.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['On']
10. Occurrences: 2445. Trajectory: [227.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
11. Occurrences: 2268. Trajectory: [226.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
12. Occurrences: 2251. Trajectory: [230.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
13. Occurrences: 620. Trajectory: [71.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['On']
14. Occurrences: 617. Trajectory: [67.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
15. Occurrences: 561. Trajectory: [70.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
16. Occurrences: 522. Trajectory: [66.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
17. Occurrences: 503. Trajectory: [16.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
18. Occurrences: 481. Trajectory: [20.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
19. Occurrences: 404. Trajectory: [8.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
20. Occurrences: 375. Trajectory: [24.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
21. Occurrences: 369. Trajectory: [12.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
22. Occurrences: 336. Trajectory: [28.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
23. Occurrences: 296. Trajectory: [110.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
24. Occurrences: 293. Trajectory: [186.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
25. Occurrences: 272. Trajectory: [190.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
26. Occurrences: 268. Trajectory: [106.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
27. Occurrences: 255. Trajectory: [178.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
28. Occurrences: 255. Trajectory: [182.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
29. Occurrences: 196. Trajectory: [147.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
30. Occurrences: 195. Trajectory: [61.,69.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
31. Occurrences: 194. Trajectory: [143.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
32. Occurrences: 194. Trajectory: [60.,65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
33. Occurrences: 192. Trajectory: [57.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
34. Occurrences: 191. Trajectory: [143.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
35. Occurrences: 189. Trajectory: [61.,65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
36. Occurrences: 188. Trajectory: [60.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
37. Occurrences: 188. Trajectory: [150.,239.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
38. Occurrences: 188. Trajectory: [61.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
39. Occurrences: 187. Trajectory: [60.,69.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
40. Occurrences: 186. Trajectory: [61.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
41. Occurrences: 186. Trajectory: [61.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
42. Occurrences: 183. Trajectory: [56.,68.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
43. Occurrences: 182. Trajectory: [151.,235.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
44. Occurrences: 182. Trajectory: [57.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
45. Occurrences: 181. Trajectory: [57.,68.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
46. Occurrences: 180. Trajectory: [57.,69.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
47. Occurrences: 179. Trajectory: [61.,68.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
48. Occurrences: 178. Trajectory: [147.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
49. Occurrences: 176. Trajectory: [151.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
50. Occurrences: 174. Trajectory: [151.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
'''

### State 56, Diabetes

In [ ]:
'''
28656
1. Occurrences: 3732. Trajectory: [69.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 3712. Trajectory: [53.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
3. Occurrences: 3442. Trajectory: [68.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
4. Occurrences: 3427. Trajectory: [52.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
5. Occurrences: 3106. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
6. Occurrences: 3040. Trajectory: [65.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
7. Occurrences: 2771. Trajectory: [48.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
8. Occurrences: 2762. Trajectory: [64.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
9. Occurrences: 2561. Trajectory: [231.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['On']
10. Occurrences: 2228. Trajectory: [230.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
11. Occurrences: 2148. Trajectory: [227.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
12. Occurrences: 2001. Trajectory: [226.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
13. Occurrences: 603. Trajectory: [71.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['On']
14. Occurrences: 563. Trajectory: [70.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
15. Occurrences: 533. Trajectory: [67.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
16. Occurrences: 484. Trajectory: [66.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
17. Occurrences: 457. Trajectory: [20.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
18. Occurrences: 416. Trajectory: [16.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
19. Occurrences: 375. Trajectory: [28.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
20. Occurrences: 375. Trajectory: [12.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
21. Occurrences: 313. Trajectory: [24.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
22. Occurrences: 308. Trajectory: [8.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
23. Occurrences: 304. Trajectory: [106.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
24. Occurrences: 289. Trajectory: [110.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
25. Occurrences: 240. Trajectory: [182.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
26. Occurrences: 236. Trajectory: [190.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
27. Occurrences: 222. Trajectory: [178.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
28. Occurrences: 201. Trajectory: [186.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
29. Occurrences: 193. Trajectory: [61.,65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
30. Occurrences: 188. Trajectory: [61.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
31. Occurrences: 183. Trajectory: [142.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
32. Occurrences: 183. Trajectory: [61.,68.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
33. Occurrences: 181. Trajectory: [61.,69.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
34. Occurrences: 180. Trajectory: [61.,64.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
35. Occurrences: 178. Trajectory: [151.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
36. Occurrences: 176. Trajectory: [60.,65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
37. Occurrences: 175. Trajectory: [151.,235.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
38. Occurrences: 175. Trajectory: [60.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
39. Occurrences: 175. Trajectory: [61.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
40. Occurrences: 175. Trajectory: [150.,230.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
41. Occurrences: 171. Trajectory: [60.,69.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
42. Occurrences: 170. Trajectory: [151.,234.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
43. Occurrences: 169. Trajectory: [151.,239.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
44. Occurrences: 168. Trajectory: [60.,64.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
45. Occurrences: 168. Trajectory: [146.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
46. Occurrences: 167. Trajectory: [143.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
47. Occurrences: 166. Trajectory: [143.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
48. Occurrences: 165. Trajectory: [60.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
49. Occurrences: 164. Trajectory: [61.,48.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
50. Occurrences: 163. Trajectory: [151.,238.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
'''

### State 56, Non Diabetes

In [ ]:
'''
73256
1. Occurrences: 2460. Trajectory: [53.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 2204. Trajectory: [52.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
3. Occurrences: 2086. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
4. Occurrences: 1788. Trajectory: [48.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
5. Occurrences: 957. Trajectory: [20.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
6. Occurrences: 827. Trajectory: [16.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
7. Occurrences: 392. Trajectory: [22.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
8. Occurrences: 324. Trajectory: [18.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
9. Occurrences: 254. Trajectory: [61.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
10. Occurrences: 243. Trajectory: [12.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
11. Occurrences: 240. Trajectory: [61.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
12. Occurrences: 220. Trajectory: [60.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
13. Occurrences: 218. Trajectory: [61.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
14. Occurrences: 212. Trajectory: [61.,48.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
15. Occurrences: 211. Trajectory: [60.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
16. Occurrences: 203. Trajectory: [8.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
17. Occurrences: 200. Trajectory: [57.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
18. Occurrences: 196. Trajectory: [60.,48.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
19. Occurrences: 190. Trajectory: [57.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
20. Occurrences: 189. Trajectory: [60.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
21. Occurrences: 178. Trajectory: [56.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
22. Occurrences: 174. Trajectory: [56.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
23. Occurrences: 163. Trajectory: [57.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
24. Occurrences: 155. Trajectory: [56.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
25. Occurrences: 151. Trajectory: [57.,48.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
26. Occurrences: 138. Trajectory: [56.,48.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
27. Occurrences: 111. Trajectory: [60.,16.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
28. Occurrences: 106. Trajectory: [61.,20.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
29. Occurrences: 102. Trajectory: [376.] Num abnormal of last state: 0
HRs : ['N ']
SBPs: ['N ']
POs : ['N ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
30. Occurrences: 100. Trajectory: [143.,178.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
31. Occurrences: 98. Trajectory: [60.,20.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
32. Occurrences: 96. Trajectory: [63.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
33. Occurrences: 91. Trajectory: [61.,16.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
34. Occurrences: 90. Trajectory: [62.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
35. Occurrences: 88. Trajectory: [59.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
36. Occurrences: 81. Trajectory: [143.,182.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
37. Occurrences: 80. Trajectory: [62.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
38. Occurrences: 80. Trajectory: [57.,20.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
39. Occurrences: 79. Trajectory: [58.,48.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
40. Occurrences: 78. Trajectory: [62.,48.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
41. Occurrences: 76. Trajectory: [59.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
42. Occurrences: 75. Trajectory: [63.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
43. Occurrences: 74. Trajectory: [142.,178.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
44. Occurrences: 72. Trajectory: [138.,182.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
45. Occurrences: 71. Trajectory: [63.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
46. Occurrences: 71. Trajectory: [139.,182.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
47. Occurrences: 71. Trajectory: [58.,49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
48. Occurrences: 71. Trajectory: [58.,52.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
49. Occurrences: 69. Trajectory: [58.,53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
50. Occurrences: 69. Trajectory: [63.,48.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
'''

### State 377, Diabetes

In [ ]:
'''
82281
1. Occurrences: 2967. Trajectory: [376.] Num abnormal of last state: 0
HRs : ['N ']
SBPs: ['N ']
POs : ['N ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
2. Occurrences: 584. Trajectory: [430.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
3. Occurrences: 573. Trajectory: [707.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
4. Occurrences: 538. Trajectory: [227.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
5. Occurrences: 535. Trajectory: [226.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
6. Occurrences: 512. Trajectory: [706.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
7. Occurrences: 414. Trajectory: [426.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
8. Occurrences: 199. Trajectory: [381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
9. Occurrences: 147. Trajectory: [380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
10. Occurrences: 113. Trajectory: [389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['H ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
11. Occurrences: 99. Trajectory: [388.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['H ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
12. Occurrences: 99. Trajectory: [373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['L ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
13. Occurrences: 92. Trajectory: [372.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['L ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
14. Occurrences: 83. Trajectory: [377.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
15. Occurrences: 59. Trajectory: [186.] Num abnormal of last state: 4
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
16. Occurrences: 56. Trajectory: [385.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['H ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
17. Occurrences: 53. Trajectory: [178.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
18. Occurrences: 50. Trajectory: [225.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
19. Occurrences: 47. Trajectory: [369.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['L ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
20. Occurrences: 46. Trajectory: [224.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
21. Occurrences: 45. Trajectory: [658.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
22. Occurrences: 45. Trajectory: [529.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
23. Occurrences: 45. Trajectory: [65.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
24. Occurrences: 45. Trajectory: [666.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
25. Occurrences: 44. Trajectory: [384.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['H ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
26. Occurrences: 44. Trajectory: [528.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
27. Occurrences: 42. Trajectory: [689.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
28. Occurrences: 41. Trajectory: [368.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['L ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
29. Occurrences: 40. Trajectory: [705.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
30. Occurrences: 39. Trajectory: [219.,226.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
31. Occurrences: 39. Trajectory: [470.,706.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
32. Occurrences: 38. Trajectory: [209.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
33. Occurrences: 37. Trajectory: [104.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
34. Occurrences: 36. Trajectory: [381.,430.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
35. Occurrences: 36. Trajectory: [219.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
36. Occurrences: 36. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
37. Occurrences: 36. Trajectory: [470.,430.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['H ', 'H ']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
38. Occurrences: 36. Trajectory: [64.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
39. Occurrences: 35. Trajectory: [462.,426.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
40. Occurrences: 35. Trajectory: [459.,227.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
41. Occurrences: 35. Trajectory: [424.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
42. Occurrences: 35. Trajectory: [545.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
43. Occurrences: 35. Trajectory: [584.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['N ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
44. Occurrences: 34. Trajectory: [471.,706.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
45. Occurrences: 34. Trajectory: [218.,231.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
46. Occurrences: 34. Trajectory: [471.,707.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
47. Occurrences: 34. Trajectory: [470.,707.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
48. Occurrences: 33. Trajectory: [48.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
49. Occurrences: 33. Trajectory: [466.,715.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
50. Occurrences: 33. Trajectory: [544.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
'''

### State 377, No Diabetes

In [ ]:
'''
86411
1. Occurrences: 5759. Trajectory: [376.] Num abnormal of last state: 0
HRs : ['N ']
SBPs: ['N ']
POs : ['N ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
2. Occurrences: 711. Trajectory: [381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
3. Occurrences: 628. Trajectory: [380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
4. Occurrences: 327. Trajectory: [377.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
5. Occurrences: 256. Trajectory: [383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
6. Occurrences: 219. Trajectory: [379.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
7. Occurrences: 209. Trajectory: [382.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
8. Occurrences: 184. Trajectory: [378.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
9. Occurrences: 86. Trajectory: [658.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
10. Occurrences: 82. Trajectory: [178.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
11. Occurrences: 72. Trajectory: [463.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['H ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
12. Occurrences: 67. Trajectory: [381.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of']
13. Occurrences: 66. Trajectory: [380.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['Of', 'On', 'Of']
14. Occurrences: 66. Trajectory: [381.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
15. Occurrences: 62. Trajectory: [462.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['H ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
16. Occurrences: 59. Trajectory: [380.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['Of', 'Of', 'Of']
17. Occurrences: 49. Trajectory: [377.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
18. Occurrences: 46. Trajectory: [459.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['H ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
19. Occurrences: 46. Trajectory: [458.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['H ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
20. Occurrences: 43. Trajectory: [381.,377.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
21. Occurrences: 43. Trajectory: [377.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'Of', 'Of']
22. Occurrences: 36. Trajectory: [463.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'Of', 'Of']
23. Occurrences: 35. Trajectory: [380.,377.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['Of', 'On', 'Of']
24. Occurrences: 31. Trajectory: [373.,414.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['L ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
25. Occurrences: 30. Trajectory: [49.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
26. Occurrences: 29. Trajectory: [463.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
27. Occurrences: 29. Trajectory: [462.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['Of', 'On', 'Of']
28. Occurrences: 28. Trajectory: [528.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
29. Occurrences: 28. Trajectory: [381.,379.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'Of', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['On', 'On', 'Of']
30. Occurrences: 27. Trajectory: [382.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['Of', 'On', 'Of']
31. Occurrences: 27. Trajectory: [458.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['Of', 'Of', 'Of']
32. Occurrences: 25. Trajectory: [332.,414.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['L ', 'L ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
33. Occurrences: 25. Trajectory: [373.,691.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
34. Occurrences: 24. Trajectory: [463.,689.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
35. Occurrences: 24. Trajectory: [377.,377.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'Of', 'Of']
VA  : ['Of', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
36. Occurrences: 23. Trajectory: [459.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
37. Occurrences: 23. Trajectory: [383.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
38. Occurrences: 23. Trajectory: [372.,410.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['N ', 'L ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
39. Occurrences: 22. Trajectory: [568.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
40. Occurrences: 22. Trajectory: [459.,689.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
41. Occurrences: 22. Trajectory: [462.,380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['Of', 'Of', 'Of']
42. Occurrences: 22. Trajectory: [462.,688.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
43. Occurrences: 22. Trajectory: [408.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
44. Occurrences: 22. Trajectory: [248.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
45. Occurrences: 22. Trajectory: [380.,382.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'Of', 'Of']
46. Occurrences: 21. Trajectory: [380.,383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'On', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['Of', 'On', 'Of']
47. Occurrences: 21. Trajectory: [529.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
48. Occurrences: 20. Trajectory: [381.,378.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['On', 'Of', 'Of']
VA  : ['Of', 'On', 'Of']
VE  : ['On', 'Of', 'Of']
49. Occurrences: 20. Trajectory: [608.,691.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
50. Occurrences: 20. Trajectory: [372.,690.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
'''

### State 696, Diabetes

In [ ]:
'''
48334
1. Occurrences: 5693. Trajectory: [707.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
2. Occurrences: 5051. Trajectory: [706.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
3. Occurrences: 3040. Trajectory: [689.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
4. Occurrences: 3030. Trajectory: [705.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
5. Occurrences: 2984. Trajectory: [711.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['On']
6. Occurrences: 2764. Trajectory: [704.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
7. Occurrences: 2763. Trajectory: [688.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
8. Occurrences: 2671. Trajectory: [710.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
9. Occurrences: 1019. Trajectory: [693.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
10. Occurrences: 913. Trajectory: [709.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
11. Occurrences: 845. Trajectory: [708.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
12. Occurrences: 839. Trajectory: [692.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
13. Occurrences: 560. Trajectory: [666.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
14. Occurrences: 557. Trajectory: [658.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
15. Occurrences: 408. Trajectory: [656.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
16. Occurrences: 317. Trajectory: [699.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
17. Occurrences: 303. Trajectory: [698.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
18. Occurrences: 301. Trajectory: [664.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
19. Occurrences: 291. Trajectory: [430.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
20. Occurrences: 289. Trajectory: [662.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
21. Occurrences: 288. Trajectory: [670.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
22. Occurrences: 275. Trajectory: [648.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
23. Occurrences: 262. Trajectory: [699.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
24. Occurrences: 258. Trajectory: [698.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
25. Occurrences: 232. Trajectory: [696.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
26. Occurrences: 228. Trajectory: [697.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
27. Occurrences: 219. Trajectory: [699.,705.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
28. Occurrences: 219. Trajectory: [697.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
29. Occurrences: 195. Trajectory: [699.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
30. Occurrences: 186. Trajectory: [703.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
31. Occurrences: 181. Trajectory: [696.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
32. Occurrences: 178. Trajectory: [698.,711.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
33. Occurrences: 177. Trajectory: [699.,711.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
34. Occurrences: 173. Trajectory: [702.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
35. Occurrences: 172. Trajectory: [703.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
36. Occurrences: 172. Trajectory: [699.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
37. Occurrences: 163. Trajectory: [699.,704.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
38. Occurrences: 160. Trajectory: [698.,705.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
39. Occurrences: 150. Trajectory: [698.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
40. Occurrences: 143. Trajectory: [702.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
41. Occurrences: 138. Trajectory: [698.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
42. Occurrences: 134. Trajectory: [697.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
43. Occurrences: 131. Trajectory: [697.,711.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
44. Occurrences: 131. Trajectory: [697.,705.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
45. Occurrences: 129. Trajectory: [698.,704.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
46. Occurrences: 129. Trajectory: [699.,710.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
47. Occurrences: 124. Trajectory: [698.,710.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
48. Occurrences: 122. Trajectory: [660.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
49. Occurrences: 119. Trajectory: [696.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
50. Occurrences: 119. Trajectory: [696.,705.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
'''


### State 696, No Diabetes

In [ ]:
'''
82124
1. Occurrences: 2030. Trajectory: [689.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 1845. Trajectory: [688.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
3. Occurrences: 1121. Trajectory: [658.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
4. Occurrences: 787. Trajectory: [656.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
5. Occurrences: 648. Trajectory: [693.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
6. Occurrences: 583. Trajectory: [662.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
7. Occurrences: 555. Trajectory: [692.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
8. Occurrences: 239. Trajectory: [660.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
9. Occurrences: 231. Trajectory: [699.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
10. Occurrences: 223. Trajectory: [648.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
11. Occurrences: 198. Trajectory: [699.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
12. Occurrences: 197. Trajectory: [381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
13. Occurrences: 189. Trajectory: [698.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
14. Occurrences: 175. Trajectory: [698.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
15. Occurrences: 175. Trajectory: [697.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
16. Occurrences: 171. Trajectory: [380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
17. Occurrences: 167. Trajectory: [696.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
18. Occurrences: 141. Trajectory: [696.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
19. Occurrences: 138. Trajectory: [697.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
20. Occurrences: 133. Trajectory: [703.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
21. Occurrences: 117. Trajectory: [698.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
22. Occurrences: 116. Trajectory: [699.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
23. Occurrences: 113. Trajectory: [703.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
24. Occurrences: 97. Trajectory: [702.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
25. Occurrences: 96. Trajectory: [376.] Num abnormal of last state: 0
HRs : ['N ']
SBPs: ['N ']
POs : ['N ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
26. Occurrences: 95. Trajectory: [702.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
27. Occurrences: 95. Trajectory: [696.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
28. Occurrences: 95. Trajectory: [697.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
29. Occurrences: 90. Trajectory: [698.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
30. Occurrences: 88. Trajectory: [699.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
31. Occurrences: 87. Trajectory: [697.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
32. Occurrences: 77. Trajectory: [699.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
33. Occurrences: 74. Trajectory: [652.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
34. Occurrences: 67. Trajectory: [696.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
35. Occurrences: 66. Trajectory: [613.,691.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
36. Occurrences: 65. Trajectory: [699.,662.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
37. Occurrences: 64. Trajectory: [572.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['N ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
38. Occurrences: 64. Trajectory: [703.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
39. Occurrences: 61. Trajectory: [412.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
40. Occurrences: 59. Trajectory: [701.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
41. Occurrences: 59. Trajectory: [702.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
42. Occurrences: 58. Trajectory: [697.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
43. Occurrences: 57. Trajectory: [698.,662.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
44. Occurrences: 55. Trajectory: [383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
45. Occurrences: 55. Trajectory: [701.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
46. Occurrences: 54. Trajectory: [382.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
47. Occurrences: 53. Trajectory: [612.,690.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
48. Occurrences: 51. Trajectory: [703.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
49. Occurrences: 49. Trajectory: [700.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
50. Occurrences: 49. Trajectory: [696.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
'''

### State 703, Diabetes

In [ ]:
'''
44828
1. Occurrences: 6300. Trajectory: [707.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
2. Occurrences: 5588. Trajectory: [706.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
3. Occurrences: 3585. Trajectory: [689.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
4. Occurrences: 3572. Trajectory: [705.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
5. Occurrences: 3207. Trajectory: [704.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
6. Occurrences: 3130. Trajectory: [688.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
7. Occurrences: 2862. Trajectory: [711.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['On']
8. Occurrences: 2607. Trajectory: [710.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
9. Occurrences: 941. Trajectory: [693.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
10. Occurrences: 881. Trajectory: [709.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
11. Occurrences: 804. Trajectory: [708.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
12. Occurrences: 781. Trajectory: [692.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
13. Occurrences: 658. Trajectory: [666.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
14. Occurrences: 640. Trajectory: [658.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
15. Occurrences: 524. Trajectory: [656.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
16. Occurrences: 349. Trajectory: [664.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
17. Occurrences: 342. Trajectory: [698.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
18. Occurrences: 328. Trajectory: [699.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
19. Occurrences: 323. Trajectory: [699.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
20. Occurrences: 309. Trajectory: [648.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
21. Occurrences: 293. Trajectory: [670.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
22. Occurrences: 280. Trajectory: [430.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
23. Occurrences: 278. Trajectory: [662.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
24. Occurrences: 266. Trajectory: [697.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
25. Occurrences: 262. Trajectory: [698.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
26. Occurrences: 239. Trajectory: [697.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
27. Occurrences: 236. Trajectory: [696.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
28. Occurrences: 216. Trajectory: [696.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
29. Occurrences: 216. Trajectory: [699.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
30. Occurrences: 199. Trajectory: [699.,705.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
31. Occurrences: 194. Trajectory: [703.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
32. Occurrences: 187. Trajectory: [698.,711.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'On']
33. Occurrences: 179. Trajectory: [699.,704.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
34. Occurrences: 179. Trajectory: [698.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
35. Occurrences: 175. Trajectory: [703.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
36. Occurrences: 174. Trajectory: [699.,710.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
37. Occurrences: 174. Trajectory: [698.,705.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
38. Occurrences: 171. Trajectory: [699.,711.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'On']
39. Occurrences: 167. Trajectory: [699.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
40. Occurrences: 160. Trajectory: [698.,710.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
41. Occurrences: 157. Trajectory: [698.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
42. Occurrences: 155. Trajectory: [698.,704.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
43. Occurrences: 149. Trajectory: [697.,704.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
44. Occurrences: 140. Trajectory: [702.,707.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'On']
45. Occurrences: 138. Trajectory: [697.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
46. Occurrences: 138. Trajectory: [696.,705.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
47. Occurrences: 137. Trajectory: [697.,711.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'On']
48. Occurrences: 136. Trajectory: [697.,705.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
49. Occurrences: 134. Trajectory: [697.,710.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
50. Occurrences: 131. Trajectory: [702.,706.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
'''

### State 703, No Diabetes

In [ ]:
'''
81086
1. Occurrences: 2273. Trajectory: [689.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
2. Occurrences: 1974. Trajectory: [688.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
3. Occurrences: 1263. Trajectory: [658.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
4. Occurrences: 916. Trajectory: [656.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
5. Occurrences: 560. Trajectory: [693.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
6. Occurrences: 533. Trajectory: [662.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
7. Occurrences: 484. Trajectory: [692.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
8. Occurrences: 259. Trajectory: [699.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
9. Occurrences: 242. Trajectory: [648.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
10. Occurrences: 233. Trajectory: [698.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
11. Occurrences: 219. Trajectory: [699.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
12. Occurrences: 215. Trajectory: [698.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
13. Occurrences: 212. Trajectory: [660.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['N ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
14. Occurrences: 189. Trajectory: [381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
15. Occurrences: 181. Trajectory: [697.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
16. Occurrences: 179. Trajectory: [380.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
17. Occurrences: 168. Trajectory: [696.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'On']
18. Occurrences: 163. Trajectory: [697.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
19. Occurrences: 148. Trajectory: [696.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
20. Occurrences: 142. Trajectory: [698.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
21. Occurrences: 130. Trajectory: [703.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
22. Occurrences: 127. Trajectory: [699.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
23. Occurrences: 119. Trajectory: [699.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
24. Occurrences: 118. Trajectory: [703.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
25. Occurrences: 113. Trajectory: [696.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
26. Occurrences: 99. Trajectory: [702.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
27. Occurrences: 96. Trajectory: [702.,688.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
28. Occurrences: 94. Trajectory: [697.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
29. Occurrences: 89. Trajectory: [697.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
30. Occurrences: 89. Trajectory: [698.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
31. Occurrences: 79. Trajectory: [699.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
32. Occurrences: 76. Trajectory: [698.,692.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
33. Occurrences: 69. Trajectory: [698.,662.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
34. Occurrences: 68. Trajectory: [652.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
35. Occurrences: 67. Trajectory: [613.,691.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
36. Occurrences: 65. Trajectory: [699.,662.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
37. Occurrences: 65. Trajectory: [412.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
38. Occurrences: 63. Trajectory: [702.,658.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
39. Occurrences: 62. Trajectory: [383.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
40. Occurrences: 60. Trajectory: [696.,656.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'Of']
VA  : ['Of', 'Of']
VE  : ['Of', 'Of']
41. Occurrences: 59. Trajectory: [698.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['Of', 'On']
42. Occurrences: 58. Trajectory: [697.,692.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'Of']
43. Occurrences: 57. Trajectory: [699.,692.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
44. Occurrences: 53. Trajectory: [533.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
45. Occurrences: 53. Trajectory: [701.,689.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
46. Occurrences: 53. Trajectory: [532.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['L ']
POs : ['N ']
GL  : ['L ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
47. Occurrences: 53. Trajectory: [612.,691.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['N ', 'H ']
POs : ['N ', 'N ']
GL  : ['L ', 'L ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
48. Occurrences: 52. Trajectory: [382.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ']
SBPs: ['N ', 'N ']
POs : ['N ', 'N ']
GL  : ['N ', 'N ']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['Of', 'Of']
49. Occurrences: 52. Trajectory: [696.,662.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['N ', 'N ']
AB  : ['Of', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
50. Occurrences: 51. Trajectory: [697.,693.] Num abnormal of last state: 3
HRs : ['H ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['Of', 'Of']
VE  : ['On', 'On']
'''

### State 439, Diabetes

In [ ]:
'''
33957
1. Occurrences: 12058. Trajectory: [438.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
2. Occurrences: 11242. Trajectory: [434.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
3. Occurrences: 7393. Trajectory: [432.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
4. Occurrences: 4213. Trajectory: [436.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
5. Occurrences: 3677. Trajectory: [439.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['On']
VE  : ['On']
6. Occurrences: 3348. Trajectory: [435.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
7. Occurrences: 3202. Trajectory: [424.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
8. Occurrences: 2246. Trajectory: [433.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
9. Occurrences: 1725. Trajectory: [428.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
10. Occurrences: 1266. Trajectory: [674.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
11. Occurrences: 1256. Trajectory: [437.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
12. Occurrences: 974. Trajectory: [425.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
13. Occurrences: 867. Trajectory: [672.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
14. Occurrences: 864. Trajectory: [715.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
15. Occurrences: 545. Trajectory: [713.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
16. Occurrences: 528. Trajectory: [429.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
17. Occurrences: 469. Trajectory: [356.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
18. Occurrences: 428. Trajectory: [356.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
19. Occurrences: 390. Trajectory: [664.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
20. Occurrences: 383. Trajectory: [675.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
21. Occurrences: 254. Trajectory: [673.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
22. Occurrences: 241. Trajectory: [705.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
23. Occurrences: 219. Trajectory: [276.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
24. Occurrences: 154. Trajectory: [356.,439.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
25. Occurrences: 153. Trajectory: [357.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
26. Occurrences: 145. Trajectory: [356.,435.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
27. Occurrences: 142. Trajectory: [357.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
28. Occurrences: 115. Trajectory: [475.,715.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
29. Occurrences: 115. Trajectory: [665.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
30. Occurrences: 115. Trajectory: [475.,234.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
31. Occurrences: 112. Trajectory: [479.,714.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
32. Occurrences: 101. Trajectory: [348.,430.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['H ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
33. Occurrences: 99. Trajectory: [348.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['H ', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
34. Occurrences: 97. Trajectory: [479.,715.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
35. Occurrences: 95. Trajectory: [475.,235.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
36. Occurrences: 94. Trajectory: [479.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
37. Occurrences: 91. Trajectory: [479.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
38. Occurrences: 91. Trajectory: [268.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
39. Occurrences: 90. Trajectory: [348.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['H ', 'VH']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
40. Occurrences: 88. Trajectory: [348.,426.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
41. Occurrences: 88. Trajectory: [475.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['VH', 'VH']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
42. Occurrences: 86. Trajectory: [475.,714.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
43. Occurrences: 77. Trajectory: [473.,235.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
44. Occurrences: 71. Trajectory: [358.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
45. Occurrences: 71. Trajectory: [475.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
46. Occurrences: 65. Trajectory: [277.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
47. Occurrences: 65. Trajectory: [479.,713.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
48. Occurrences: 62. Trajectory: [479.,712.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
49. Occurrences: 60. Trajectory: [473.,715.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
50. Occurrences: 59. Trajectory: [475.,232.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
'''

### State 439 No Diabetes

In [ ]:
'''
33957
1. Occurrences: 12058. Trajectory: [438.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['On']
VE  : ['Of']
2. Occurrences: 11242. Trajectory: [434.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
3. Occurrences: 7393. Trajectory: [432.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
4. Occurrences: 4213. Trajectory: [436.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
5. Occurrences: 3677. Trajectory: [439.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['On']
VE  : ['On']
6. Occurrences: 3348. Trajectory: [435.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
7. Occurrences: 3202. Trajectory: [424.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
8. Occurrences: 2246. Trajectory: [433.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
9. Occurrences: 1725. Trajectory: [428.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
10. Occurrences: 1266. Trajectory: [674.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['Of']
11. Occurrences: 1256. Trajectory: [437.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
12. Occurrences: 974. Trajectory: [425.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
13. Occurrences: 867. Trajectory: [672.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
14. Occurrences: 864. Trajectory: [715.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
15. Occurrences: 545. Trajectory: [713.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
16. Occurrences: 528. Trajectory: [429.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
17. Occurrences: 469. Trajectory: [356.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
18. Occurrences: 428. Trajectory: [356.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
19. Occurrences: 390. Trajectory: [664.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['Of']
20. Occurrences: 383. Trajectory: [675.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
21. Occurrences: 254. Trajectory: [673.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['VH']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
22. Occurrences: 241. Trajectory: [705.] Num abnormal of last state: 3
HRs : ['H ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
23. Occurrences: 219. Trajectory: [276.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
24. Occurrences: 154. Trajectory: [356.,439.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
25. Occurrences: 153. Trajectory: [357.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
26. Occurrences: 145. Trajectory: [356.,435.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'On']
27. Occurrences: 142. Trajectory: [357.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'Of']
28. Occurrences: 115. Trajectory: [475.,715.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
29. Occurrences: 115. Trajectory: [665.] Num abnormal of last state: 4
HRs : ['H ']
SBPs: ['H ']
POs : ['L ']
GL  : ['H ']
AB  : ['Of']
VA  : ['Of']
VE  : ['On']
30. Occurrences: 115. Trajectory: [475.,234.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
31. Occurrences: 112. Trajectory: [479.,714.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
32. Occurrences: 101. Trajectory: [348.,430.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['H ', 'H ']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
33. Occurrences: 99. Trajectory: [348.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['H ', 'VH']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
34. Occurrences: 97. Trajectory: [479.,715.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
35. Occurrences: 95. Trajectory: [475.,235.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
36. Occurrences: 94. Trajectory: [479.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
37. Occurrences: 91. Trajectory: [479.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
38. Occurrences: 91. Trajectory: [268.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['H ']
AB  : ['On']
VA  : ['Of']
VE  : ['Of']
39. Occurrences: 90. Trajectory: [348.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['H ', 'VH']
AB  : ['On', 'On']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
40. Occurrences: 88. Trajectory: [348.,426.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['H ', 'H ']
AB  : ['On', 'Of']
VA  : ['Of', 'On']
VE  : ['Of', 'Of']
41. Occurrences: 88. Trajectory: [475.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['VH', 'VH']
AB  : ['Of', 'On']
VA  : ['On', 'On']
VE  : ['On', 'Of']
42. Occurrences: 86. Trajectory: [475.,714.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
43. Occurrences: 77. Trajectory: [473.,235.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
44. Occurrences: 71. Trajectory: [358.,438.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['N ', 'H ']
POs : ['L ', 'L ']
GL  : ['VH', 'VH']
AB  : ['On', 'On']
VA  : ['On', 'On']
VE  : ['Of', 'Of']
45. Occurrences: 71. Trajectory: [475.,434.] Num abnormal of last state: 3
HRs : ['N ', 'N ']
SBPs: ['H ', 'H ']
POs : ['N ', 'L ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'Of']
46. Occurrences: 65. Trajectory: [277.] Num abnormal of last state: 3
HRs : ['N ']
SBPs: ['L ']
POs : ['L ']
GL  : ['VH']
AB  : ['On']
VA  : ['Of']
VE  : ['On']
47. Occurrences: 65. Trajectory: [479.,713.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
48. Occurrences: 62. Trajectory: [479.,712.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['On', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
49. Occurrences: 60. Trajectory: [473.,715.] Num abnormal of last state: 3
HRs : ['N ', 'H ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['Of', 'On']
VE  : ['On', 'On']
50. Occurrences: 59. Trajectory: [475.,232.] Num abnormal of last state: 3
HRs : ['N ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['VH', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'Of']
'''